In [8]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# aqui eu deixo fixo porque o dataset é de 2019 (igual no material)
ANO = "2019"

# eu criei esse dicionário só pra converter a sigla do mês em número
MAP_MES_NUM = {
    "JAN": "01", "FEV": "02", "MAR": "03", "ABR": "04", "MAI": "05", "JUN": "06",
    "JUL": "07", "AGO": "08", "SET": "09", "OUT": "10", "NOV": "11", "DEZ": "12"
}

def garantir_pasta(pasta: Path) -> None:
    # aqui eu garanto que a pasta existe (se não existir, eu crio)
    pasta.mkdir(parents=True, exist_ok=True)

def achar_arquivo_mes(pasta_input: Path, sigla_mes: str) -> Path:
    # aqui eu tento encontrar o arquivo tanto com .csv quanto sem extensão 
    candidatos = [
        pasta_input / f"SINASC_RO_{ANO}_{sigla_mes}.csv",
        pasta_input / f"SINASC_RO_{ANO}_{sigla_mes}",
    ]
    for arq in candidatos:
        if arq.exists():
            return arq
    raise FileNotFoundError(f"Não achei o arquivo do mês {sigla_mes} dentro de {pasta_input}.")

def salvar_fig(pasta: Path, nome: str) -> None:
    # aqui eu salvo e fecho a figura pra não acumular memória
    plt.savefig(pasta / nome, dpi=200, bbox_inches="tight")
    plt.close()

def preparar_dtnasc(df: pd.DataFrame) -> pd.DataFrame:
    # aqui eu tento transformar a DTNASC em data (pra ficar certinho no eixo x)
    if "DTNASC" in df.columns:
        df["DTNASC"] = pd.to_datetime(df["DTNASC"], errors="coerce")
    return df

def gerar_graficos_mes(df: pd.DataFrame, pasta_saida: Path, sigla_mes: str) -> None:
    df = preparar_dtnasc(df)

    # 1) média da idade da mãe ao longo do tempo
    if {"DTNASC", "IDADEMAE"}.issubset(df.columns):
        serie = df.dropna(subset=["DTNASC", "IDADEMAE"]).groupby("DTNASC")["IDADEMAE"].mean().sort_index()
        plt.figure(figsize=(16, 5))
        plt.plot(serie.index, serie.values)
        plt.title("Média da idade da mãe ao longo do tempo")
        plt.xlabel("data de nascimento")
        plt.ylabel("média idade mãe")
        salvar_fig(pasta_saida, f"{sigla_mes}_1_media_idade_mae.png")

    # 2) quantidade de bebês ao longo do tempo (contagem)
    if "DTNASC" in df.columns:
        cont = df.dropna(subset=["DTNASC"]).groupby("DTNASC").size().sort_index()
        plt.figure(figsize=(16, 5))
        plt.plot(cont.index, cont.values)
        plt.title("Quantidade de bebês ao longo do tempo")
        plt.xlabel("data de nascimento")
        plt.ylabel("quantidade de nascimentos")
        salvar_fig(pasta_saida, f"{sigla_mes}_2_qtd_bebes.png")

    # 3) quantidade de bebês fem/masc por dia
    if {"DTNASC", "SEXO"}.issubset(df.columns):
        tab = df.dropna(subset=["DTNASC", "SEXO"]).groupby(["DTNASC", "SEXO"]).size().unstack(fill_value=0).sort_index()
        plt.figure(figsize=(16, 6))
        tab.plot(kind="line", figsize=(16, 6))
        plt.title("Quantidade de bebês feminino e masculino por dia")
        plt.xlabel("data de nascimento")
        plt.ylabel("quantidade de nascimentos")
        salvar_fig(pasta_saida, f"{sigla_mes}_3_qtd_bebes_sexo.png")

    # 4) média do peso (fem/masc)
    if {"DTNASC", "SEXO", "PESO"}.issubset(df.columns):
        tab = df.dropna(subset=["DTNASC", "SEXO", "PESO"]).groupby(["DTNASC", "SEXO"])["PESO"].mean().unstack().sort_index()
        plt.figure(figsize=(16, 6))
        tab.plot(kind="line", figsize=(16, 6))
        plt.title("Média do peso dos bebês (fem e masc)")
        plt.xlabel("data de nascimento")
        plt.ylabel("média peso do bebê")
        salvar_fig(pasta_saida, f"{sigla_mes}_4_media_peso_sexo.png")

    # 5) mediana do peso por escolaridade da mãe
    if {"ESCMAE", "PESO"}.issubset(df.columns):
        med = df.dropna(subset=["ESCMAE", "PESO"]).groupby("ESCMAE")["PESO"].median().sort_values()
        plt.figure(figsize=(16, 6))
        med.plot(kind="bar", figsize=(16, 6))
        plt.title("Mediana do peso por escolaridade da mãe")
        plt.xlabel("escolaridade da mãe")
        plt.ylabel("mediana peso do bebê")
        salvar_fig(pasta_saida, f"{sigla_mes}_5_mediana_peso_escmae.png")

    # 6) APGAR1 médio por gestação
    if {"GESTACAO", "APGAR1"}.issubset(df.columns):
        m = df.dropna(subset=["GESTACAO", "APGAR1"]).groupby("GESTACAO")["APGAR1"].mean().sort_values()
        plt.figure(figsize=(16, 6))
        m.plot(kind="bar", figsize=(16, 6))
        plt.title("APGAR1 médio por gestação")
        plt.xlabel("gestação")
        plt.ylabel("apgar1 médio")
        salvar_fig(pasta_saida, f"{sigla_mes}_6_apgar1_medio_gestacao.png")

    # 7) APGAR5 médio por gestação
    if {"GESTACAO", "APGAR5"}.issubset(df.columns):
        m = df.dropna(subset=["GESTACAO", "APGAR5"]).groupby("GESTACAO")["APGAR5"].mean().sort_values()
        plt.figure(figsize=(16, 6))
        m.plot(kind="bar", figsize=(16, 6))
        plt.title("APGAR5 médio por gestação")
        plt.xlabel("gestação")
        plt.ylabel("apgar5 médio")
        salvar_fig(pasta_saida, f"{sigla_mes}_7_apgar5_medio_gestacao.png")

def rodar(meses: list[str], base_input: str = "./input", base_images: str = "./imagens") -> None:
    base_input = Path(base_input)
    base_images = Path(base_images)

    garantir_pasta(base_images)

    for sigla in meses:
        sigla = sigla.strip().upper()
        if sigla not in MAP_MES_NUM:
            print(f"[ERRO] Mês inválido: {sigla}. Use tipo MAR ABR MAI...")
            continue

        mes_num = MAP_MES_NUM[sigla]
        pasta_saida = base_images / f"{ANO}-{mes_num}"
        garantir_pasta(pasta_saida)

        arq = achar_arquivo_mes(base_input, sigla)
        print(f"Processando {sigla} -> {arq.name}")

        df = pd.read_csv(arq)
        gerar_graficos_mes(df, pasta_saida, sigla)

        print(f"[OK] Gráficos salvos em: {pasta_saida}")

    print("Finalizado.")


In [9]:
rodar(["MAR", "ABR", "MAI", "JUN", "DEZ"])

Processando MAR -> SINASC_RO_2019_MAR.csv
[OK] Gráficos salvos em: imagens\2019-03
Processando ABR -> SINASC_RO_2019_ABR.csv
[OK] Gráficos salvos em: imagens\2019-04
Processando MAI -> SINASC_RO_2019_MAI.csv
[OK] Gráficos salvos em: imagens\2019-05
Processando JUN -> SINASC_RO_2019_JUN.csv
[OK] Gráficos salvos em: imagens\2019-06
Processando DEZ -> SINASC_RO_2019_DEZ.csv
[OK] Gráficos salvos em: imagens\2019-12
Finalizado.


<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>

...

<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>